# IX System Simulation - CLI Based

This notebook uses the IX CLI for all simulations, ensuring consistent results between notebook and script execution.

## Key Features:
- Single source of truth via ix_cli.py
- Process isolation for each simulation
- No duplicate simulation logic
- Deterministic results

In [ ]:
# Parameters cell - papermill will inject values here
import json
import os
from pathlib import Path

# Configuration parameters (injected by papermill)
project_root = None  # Will be injected
watertap_ix_transport_path = None  # Path to watertap_ix_transport module  
configuration = None  # IX configuration
water_analysis = None  # Feed water composition
breakthrough_criteria = None  # Breakthrough criteria dictionary
regenerant_parameters = None  # Regenerant parameters dictionary
acid_options = None  # Acid dosing options for degasser
timestamp = None  # Timestamp for the simulation run
simulation_options = {
    "solver_options": {
        "tol": 1e-6,
        "constr_viol_tol": 1e-6,
        "max_iter": 100,
        "mu_strategy": "adaptive"
    }
}

In [ ]:
# System imports and setup
import subprocess
import tempfile
import json
import logging
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Validate inputs
validation_errors = []
if not configuration:
    validation_errors.append("Missing configuration parameter")
if not water_analysis:
    validation_errors.append("Missing water_analysis parameter")
if not project_root:
    validation_errors.append("Missing project_root parameter")

if validation_errors:
    raise ValueError(f"Validation failed: {', '.join(validation_errors)}")

logger.info(f"Configuration loaded: {configuration.get('flowsheet_type', 'Unknown')}")
logger.info(f"Water analysis loaded: Flow {water_analysis.get('flow_m3_hr', 0)} m³/hr")

In [ ]:
# Prepare configuration for CLI
cli_config = {
    "water_analysis": water_analysis,
    "configuration": configuration,
    "solver_options": simulation_options.get("solver_options", {})
}

# Write config to temporary file
with tempfile.NamedTemporaryFile(mode='w', suffix='.json', delete=False) as f:
    config_path = f.name
    json.dump(cli_config, f, indent=2)

logger.info(f"Configuration written to: {config_path}")

In [ ]:
# Run simulation via CLI subprocess
logger.info("Running simulation via CLI...")

# Prepare output path
with tempfile.NamedTemporaryFile(mode='w', suffix='.json', delete=False) as f:
    output_path = f.name

# Build command
python_exe = "python" if os.name != 'nt' else "python.exe"
cli_path = Path(project_root) / "ix_cli.py"

cmd = [
    python_exe,
    str(cli_path),
    "run",
    config_path,
    "--output", output_path,
    "--verbose"
]

logger.info(f"Command: {' '.join(cmd)}")

# Run in subprocess for complete isolation
try:
    result = subprocess.run(
        cmd,
        capture_output=True,
        text=True,
        cwd=project_root,
        timeout=300  # 5 minute timeout
    )
    
    logger.info(f"CLI exit code: {result.returncode}")
    
    if result.stdout:
        logger.info("CLI stdout:")
        print(result.stdout)
    
    if result.stderr:
        logger.warning("CLI stderr:")
        print(result.stderr)
        
except subprocess.TimeoutExpired:
    logger.error("CLI simulation timed out after 5 minutes")
    raise
except Exception as e:
    logger.error(f"CLI execution failed: {str(e)}")
    raise
finally:
    # Clean up config file
    try:
        os.unlink(config_path)
    except:
        pass

In [ ]:
# Load results from CLI output
try:
    with open(output_path, 'r') as f:
        cli_results = json.load(f)
    logger.info("Results loaded successfully")
except Exception as e:
    logger.error(f"Failed to load results: {str(e)}")
    raise
finally:
    # Clean up output file
    try:
        os.unlink(output_path)
    except:
        pass

# Extract key results
simulation_status = cli_results['simulation']['status']
logger.info(f"Simulation status: {simulation_status}")

In [ ]:
# Process and format results for MCP server
if simulation_status == 'success':
    treated_water = cli_results['simulation']['treated_water']
    performance = cli_results['simulation']['performance']
    
    # Extract IX performance metrics
    ix_performance = {}
    for vessel_name in configuration['ix_vessels'].keys():
        vessel_config = configuration['ix_vessels'][vessel_name]
        
        # Estimate breakthrough time (simplified)
        bed_volume = vessel_config['resin_volume_m3']
        flow_rate = water_analysis['flow_m3_hr']
        
        # Use removal percentage to estimate capacity utilization
        avg_removal = (performance['ca_removal_percent'] + performance['mg_removal_percent']) / 2
        breakthrough_bv = int(avg_removal * 10)  # Rough estimate
        
        ix_performance[vessel_name] = {
            'breakthrough_time_hours': breakthrough_bv * bed_volume / flow_rate,
            'bed_volumes_treated': breakthrough_bv,
            'regenerant_consumption_kg': bed_volume * 120,  # Default 120 kg/m³
            'average_hardness_leakage_mg_L': treated_water['ion_concentrations_mg_L'].get('Ca_2+', 0) * 2.5 + 
                                            treated_water['ion_concentrations_mg_L'].get('Mg_2+', 0) * 4.1,
            'capacity_utilization_percent': min(100, avg_removal)
        }
    
    # Build recommendations
    recommendations = []
    if performance['ca_removal_percent'] > 90:
        recommendations.append("Excellent hardness removal achieved")
    elif performance['ca_removal_percent'] > 70:
        recommendations.append("Good hardness removal, consider optimizing regenerant dose")
    else:
        recommendations.append("Poor hardness removal, check resin condition and regeneration")
    
    if performance['mass_balance_error_percent'] > 5:
        recommendations.append(f"High mass balance error ({performance['mass_balance_error_percent']:.1f}%) - check model convergence")
    
    # Final result structure for MCP
    result = {
        'status': 'success',
        'watertap_notebook_path': 'cli_based_execution',
        'model_type': 'watertap_cli',
        'actual_runtime_seconds': cli_results['simulation']['solve_time'],
        'treated_water': treated_water,
        'ix_performance': ix_performance,
        'water_quality_progression': [],  # Could be populated from stage results
        'economics': {},  # Simplified for now
        'recommendations': recommendations,
        'detailed_results': {
            'solver_termination': cli_results['simulation']['solver_termination'],
            'degrees_of_freedom': cli_results['initialization']['dof'],
            'mass_balance_error': performance['mass_balance_error_percent']
        }
    }
else:
    # Error case
    result = {
        'status': 'error',
        'watertap_notebook_path': 'cli_based_execution',
        'model_type': 'watertap_cli',
        'actual_runtime_seconds': 0,
        'treated_water': water_analysis,
        'ix_performance': {},
        'water_quality_progression': [],
        'economics': {},
        'recommendations': [f"Simulation failed: {cli_results['simulation']['solver_termination']}"],
        'detailed_results': cli_results
    }

logger.info("Results processing complete")

In [ ]:
# Visualization of results
if simulation_status == 'success':
    # Create performance visualization
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    # Removal percentages
    ions = ['Ca²⁺', 'Mg²⁺']
    removals = [performance['ca_removal_percent'], performance['mg_removal_percent']]
    
    ax1.bar(ions, removals, color=['blue', 'green'])
    ax1.set_ylabel('Removal %')
    ax1.set_title('Ion Removal Performance')
    ax1.set_ylim(0, 100)
    
    # Concentration comparison
    feed_conc = [water_analysis['ion_concentrations_mg_L'].get('Ca_2+', 0),
                 water_analysis['ion_concentrations_mg_L'].get('Mg_2+', 0)]
    product_conc = [treated_water['ion_concentrations_mg_L'].get('Ca_2+', 0),
                    treated_water['ion_concentrations_mg_L'].get('Mg_2+', 0)]
    
    x = range(len(ions))
    width = 0.35
    
    ax2.bar([i - width/2 for i in x], feed_conc, width, label='Feed', color='red', alpha=0.7)
    ax2.bar([i + width/2 for i in x], product_conc, width, label='Product', color='green', alpha=0.7)
    ax2.set_ylabel('Concentration (mg/L)')
    ax2.set_title('Feed vs Product Water Quality')
    ax2.set_xticks(x)
    ax2.set_xticklabels(ions)
    ax2.legend()
    
    plt.tight_layout()
    plt.show()
    
    # Display key metrics
    print(f"\nKey Performance Metrics:")
    print(f"- Ca²⁺ removal: {performance['ca_removal_percent']:.1f}%")
    print(f"- Mg²⁺ removal: {performance['mg_removal_percent']:.1f}%")
    print(f"- Mass balance error: {performance['mass_balance_error_percent']:.1f}%")
    print(f"- Solver status: {cli_results['simulation']['solver_termination']}")

In [ ]:
# Results cell for papermill extraction
# This variable will be extracted by the MCP server
results = result
results